# 3D Prediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120
import sys
sys.path.insert(0, "../src")
from cut_predictor import MeshPredictor

KeyboardInterrupt: 

In [ ]:
import tensorflow
tensorflow.__version__


In [ ]:
doe = pd.read_csv('../data/doe.csv')
doe

In [ ]:
data = pd.read_csv('../data/zt_all_raw.csv')
data.drop(data[data.doe_id == 1000].index, inplace=True)
data.drop(data[data.doe_id == 247].index, inplace=True)
data

In [ ]:
reg = MeshPredictor()
reg.load_data(
    doe = doe,
    data = data,
    index='doe_id',
    process_parameters = [
        'Blechdicke', 
        'Niederhalterkraft', 
        'Ziehspalt', 
        'Einlegeposition', 
        'Ziehtiefe',
        'Stempel_ID',
        'E',
        'Rp0',
        'Rp50'
    ],
    categorical = [
        'Ziehspalt', 
        'Ziehtiefe',
        'Stempel_ID',
    ],
    position = ['x', 'y', 'z'],
    output = ['deviation']#, 'thickness']
)

In [ ]:
reg.data_summary()

In [ ]:
config = {
    'batch_size': 2048*16,
    'max_epochs': 20,
    'layers': [256, 128, 256, 256, 256],
    'dropout': 0.0,
    'learning_rate': 0.001,
    'activation': 'lrelu'
}

reg.custom_model(save_path='../models/best_3d_model', config=config, verbose=True)
reg.training_summary()

In [ ]:
reg.load_network('../models/best_3d_model')

In [ ]:
idx = np.random.choice(data['doe_id'].unique()) 
print("Doe_ID", idx)
reg.compare(idx)

In [ ]:
geometry_50 = pd.read_csv('../data/geometry_zt50.csv')

parameters = {
    'Blechdicke': 1.01, 
    'Niederhalterkraft': 410.0, 
    'Ziehspalt': 2.4, 
    'Einlegeposition': 0, 
    'Stempel_ID': 3,
    'Ziehtiefe': 50,
    'E': 191.37245,
    'Rp0': 138.22696,
    'Rp50': 449.528189,
}

positions = geometry_50[['x', 'y', 'z']].to_numpy()
predictions = reg.predict(positions, parameters)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(positions[:, 0], positions[:, -2], positions[:, -1], 
    c=predictions[:, 0], 
    cmap='seismic') 
fig.colorbar(p, ax=ax)

In [ ]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 120

positions_zt30 = pd.read_csv('../data/geometry_zt30.csv')[['x', 'y', 'z']].to_numpy()
positions_zt50 = pd.read_csv('../data/geometry_zt50.csv')[['x', 'y', 'z']].to_numpy()
positions_zt70 = pd.read_csv('../data/geometry_zt70.csv')[['x', 'y', 'z']].to_numpy()

reg.interactive(positions_zt30)

In [ ]:
import scale.fem.lsdyna.api as api
mesh = api.BoxMesh2d(size=[1,1,0], elsize=.1, pid=1)
mesh.gen_box()
mesh.info()
mesh.nodes = mesh.nodes.rename(columns={"x":"u", "y":"v"})